# Búsqueda de hiperparámetros

Sabes que para entrenar un modelo de machine learning es necesario establecer algunos valores y configuraciones para modificar el comportamiento del entrenamiento, estos son conocidos como hiperparámetros. Por poner un ejemplo de estos hiperparámetros, toma la clase <code>RandomForestRegressor</code> (más adelante veremos los algoritmos de machine learning que sklear nos ofrece, por el momento no te preocupes):

In [ ]:
from sklearn.ensemble import RandomForestRegressor

models = RandomForestRegressor(
    n_estimators = 10,
    criterion = "gini",
    max_depth = 10,
    max_leaf_nodes = 100
)

En donde los hiperarámetros son: el número de árboles, el criterio de división, la profundidad máxima y la cantidad mínima de muestras por hoja.

Estos valores tienen un impacto significativo en el desempeño del modelo, y pueden ser la diferencia entre un mal modelo y uno que funciona a la perfección.

A pesar de que los hiperparámetros por default que las clases de scikit-learn tienen son valores razonables, no son necesariamente óptimos para todos los conjuntos de datos o para todos los problemas de aprendizaje automático. Por lo tanto, es importante hacer una búsqueda de hiperparámetros para encontrar los valores óptimos que maximicen el desempeño del modelo en todos nuestros conjuntos de datos.

Realizar esta búsqueda lleva tiempo y esfuerzo, pero es una inversión que vale la pena hacer por la mejora que representan en nuestro modelo estos parámetros.

Scikit-learn nos ofrece varias opciones para cuando se trata de hacer la búsqueda de estos hiperparámetros de forma sistemática en lugar de manual.

Las técnicas son: <i>grid search</i> o búsqueda en cuadrícula y búsqueda aleatoria o <i>random search</i> en inglés. Cada una tiene sus ventajas y desventajas, en esta lección yo te hablaré de la búsqueda aleatoria:

Solo una pequeña nota, en scikit-learn las búsquedas de hiperparámetros están siempre conectadas con al validación cruzada, para garantizar que los valores elegidos sean una elección correcta para el conjunto de datos.

## Random search

Ahora si, vamos a ver un ejemplo en un problema de regresión.

Primero, carguemos el dataset y dividámoslo en los conjuntos de entrenamiento y prueba:

In [ ]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

housing_dataset = fetch_california_housing()

X_train, X_test, y_train, y_test = train_test_split(
	housing_dataset.data,
	housing_dataset.target,
	random_state=42
)

Luego vamos a crear un modelo de regresión:

In [ ]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor()

Debemos definir el espacio de parámetros en el que vamos a buscar - este espacio de búsqueda será utilizado por Random Search para generar combinaciones de hiperparámetros de forma aleatoria, estas combinaciones serán utilizadas para crear nuevas instancias de nuestro RandomForestRegressor y ejecutar validación cruzada sobre ellos, evaluando así qué tan buenos son para encontrar la mejor combinación.

In [ ]:
param_distributions = {
    # 'n_estimators': [100, 1000, 2000],
    # 'criterion': ["squared_error", "absolute_error", "friedman_mse"],
    # 'max_depth': [None, 10, 100],
    'max_features': ["sqrt", "log2"],
    'max_leaf_nodes': [None, 10, 100, 1000]
}

Y finalmente, importamos la clase <code>RandomizedSearchCV</code>:

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

Creamos una instancia, pasándole el modelo, el conjunto de parámetros. Después especificamos el número de iteraciones, recuerda que la búsqueda is aleatoria, el número de iteraciones especifica cuántos intentos haremos para encontrar los mejores hiperparámetros. Con <code>cv</code> especificamos el número de subconjuntos para la validación cruzada y por último, fijamos el estado aleatorio en 42 para que el resultado sea reproducible.

In [ ]:
search = RandomizedSearchCV(model, param_distributions, n_iter=50, cv=5, random_state=42)

Por último llamamos a <code>fit</code> para comenzar la búsqueda, este recibe los datos de entrenamiento:

In [ ]:
search.fit(X_train, y_train)

Este se va a tardar un poco, pero al terminar vamos a poder acceder a los mejores parámetros utilizando el atributo <code>best_params_</code> y podemos evaluar el mejor modelo conseguido a través del método <code>score</code>:

In [ ]:
print("Mejores hiperparámetros: ", search.best_params_)
print("Puntuación de prueba: ", search.score(X_test, y_test))

## Entrenando un modelo con los mejores parámetros

Para entrenar el modelo final, podemos tomar los mejores hiperparámetros y pasarlos al constructor, esto crea un modelo fresco con la configuración ideal que acabamos de conseguir y lo entrena con la totalidad de nuestros datos de entrenamiento:

In [ ]:
best_model = RandomForestRegressor(**search.best_params_)

best_model.fit(X_train, y_train)

 > 📚 De tarea, practica usando una búsqueda en cuadrícula, utilizando <code>GridSearchCV</code>. Cuidado con utilizar muchos parámetros porque <i>grid search</i> toma tiempo para ejecutarse.

## No garantiza la mejor solución

Es importante tener en cuenta que la búsqueda de hiperparámetros no garantiza encontrar el conjunto óptimo de hiperparámetros para un modelo dado. Es posible que la combinación óptima de hiperparámetros no se encuentre en el espacio de búsqueda especificado manualmente. Por lo tanto, es importante considerar la búsqueda de hiperparámetros como un proceso iterativo que puede requerir varias iteraciones para alcanzar un conjunto óptimo de hiperparámetros para un modelo dado.

## En conclusión

La búsqueda de hiperparámetros es un paso crucial cuando quieres sacarle el máximo provecho a los datos. En scikit-learn esta búsqueda está fuertemente ligada con la validación cruzada aunque en la teoría son dos conceptos independientes el uno del otro.

scikit-learn ofrece dos métodos de búsqueda de hiperparámetros: GridSearchCV y RandomizedSearchCV. El primero realiza una búsqueda exhaustiva sobre todas las combinaciones posibles de valores de hiperparámetros especificados, mientras que el segundo realiza una búsqueda aleatoria de un subconjunto de combinaciones. En general, RandomizedSearchCV puede ser más eficiente que GridSearchCV cuando el espacio de búsqueda de hiperparámetros es grande.

También recuerda que no es una solución mágica, y que a veces debes iterar en la elección del mejor espacio de búsqueda.